Librerias

In [27]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

Iniciar sesión

In [28]:
# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la Web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# Colocar las claves
element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

# click en entrar
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# clickear en la pestaña de "facturación"
element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

# clickear en la sub-pestaña de "facturación" (para cargar facturas de ventas)
driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button5")
element.click()
time.sleep(1)

Cambiar de frame al marco interno

In [29]:
driver.switch_to.frame('marco')

Establecer factura manual

In [30]:
# Quitar check de "aéreo"
element = driver.find_element(By.ID,"CheckBoxList1_0")
element.click()
time.sleep(2)

# Poner check en "manual"
element = driver.find_element(By.ID,"CheckBoxList1_3")
element.click()
time.sleep(2)

# Buscar el input de IVAS
element = driver.find_element(By.ID, "contar")

# Seleccionar todo el texto actual
element.send_keys(Keys.CONTROL, "a")
time.sleep(0.2)

# Escribir el nuevo valor (3 por la información que manejamos)
element.send_keys("3")

# Disparar el evento de cambio usando TAB (como si el usuario saliera del campo)
element.send_keys(Keys.TAB)
time.sleep(3)

CAMBIAR LOS IVAS

In [31]:
# Valores específicos para cada campo
valores = {
    "ivasa1": "0",
    "ivasa2": "10.5"  # con punto, ojo a la compa que rompe la página
}

for field_id, valor in valores.items():
    element = driver.find_element(By.ID, field_id)

    # Escribir directamente el nuevo valor (sin dejar el campo vacío)
    element.send_keys(Keys.CONTROL, 'a')
    element.send_keys(valor)
    element.send_keys(Keys.TAB)
    time.sleep(3)

Hasta aquí logramos que el sistema configure todo para empezar a cargar facturas....
Para que funcione, creo que debemos hacer lo siguiente:
- pasarle la lista de facturas que queremos que cargue + el csv de lo que ya existe
- que haga una comparación con las facturas que existen actualmente
- que las cargue. 

Para una primera versión, podemos hacer la comparación manual tal vez.

PASO FALTANTE: PROCESAR FACTURAS PARA OBTENER DATOS

In [32]:
import pandas as pd
import string
import numpy as np
import re

data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Emitidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['codigo_fc'] = [int(re.match(r'(\d+)', x).group(1)) for x in data['Tipo']]
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

data['tipo2_new'] = np.where(data['codigo_fc'].isin([1,6,11]), "Factura", "Credito")
data['tipo2_new'] = np.where(data['codigo_fc'] == 201, "Pyme_fc", data['tipo2_new'])
data['tipo2_new'] = np.where(data['codigo_fc'] == 203, "Pyme_nc", data['tipo2_new'])

data['tipo3_new'] = np.where(data['codigo_fc'].isin([1,3,201,203]), "A", "C")
data['tipo3_new'] = np.where(data['codigo_fc'].isin([6,8]), "B", data['tipo3_new'])

data['tipo3_new']

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Número Desde'].astype(int)

# Cliente
data['Denominación Receptor'] = data['Denominación Receptor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Cliente'] = data['Denominación Receptor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Receptor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])
data['TOTAL_10.5'] = round(data['NETO 10.5'] + data['IVA 10.5'],2)
data['TOTAL_21'] = round(data['NETO 21'] + data['IVA 21'],2)


# PONER 0 A TODAS LAS COLUMNAS QUE NO TIENEN ALGO
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 0','NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)
data['TOTAL_21'] = data['TOTAL_21'].astype(str)
data['TOTAL_10.5'] = data['TOTAL_10.5'].astype(str)

# PONER EN NEGATIVO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    mask = (data['tipo2_new'] == 'Credito') | (data['tipo2_new'] == 'Pyme_nc')
    data.loc[mask, col] = -data.loc[mask, col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','tipo2_new','tipo3_new', 'Factura',
        'Cliente','CUIT', 'TOTAL_10.5', 'TOTAL_21',
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,3237,PILISAR S A,33709180849,379961.2,30681.22,7158.85
1,01/10/2025,Factura,A,3236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,70149.06,885106.0
2,01/10/2025,Pyme_fc,A,213,PILISAR S A,33709180849,0.0,911434.46,11500025.0
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,911434.46,11500025.0
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.0,-911434.46,-11500025.0
...,...,...,...,...,...,...,...,...,...
59,17/10/2025,Factura,A,3270,NEW PGA S R L,30615265450,1094471.24,519215.38,14317.75
60,17/10/2025,Factura,B,3601,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,1089728.0
61,17/10/2025,Factura,B,3602,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,1029465.6
62,17/10/2025,Factura,B,3603,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,238530.56


In [33]:
# cambio los valores para que se convierta en un dataset de prueba para meter registros

clean_data['Factura'] = 0
clean_data['Cliente'] = "Prueba"
clean_data['TOTAL_10.5'] = 0
clean_data['TOTAL_21'] = 0
clean_data['TOTAL_NO_GRAVADO'] = 0

C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_14764\706179197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Factura'] = 0
C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_14764\706179197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Cliente'] = "Prueba"
C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_14764\706179197.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [34]:
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,0,Prueba,33709180849,0,0,0
1,01/10/2025,Factura,A,0,Prueba,30642617555,0,0,0
2,01/10/2025,Pyme_fc,A,0,Prueba,33709180849,0,0,0
3,01/10/2025,Pyme_fc,A,0,Prueba,30642617555,0,0,0
4,01/10/2025,Pyme_nc,A,0,Prueba,30642617555,0,0,0
...,...,...,...,...,...,...,...,...,...
59,17/10/2025,Factura,A,0,Prueba,30615265450,0,0,0
60,17/10/2025,Factura,B,0,Prueba,33648976289,0,0,0
61,17/10/2025,Factura,B,0,Prueba,33648976289,0,0,0
62,17/10/2025,Factura,B,0,Prueba,33648976289,0,0,0


Hemos logrado crear un dataset con todas las facturas que tiene el sistema, ahora solo falta que las cargue.


De aquí hacia abajo no funciona la cosa

----------------------



In [35]:
for index, row in clean_data.iterrows():
    # TOTAL_NO_GRAVADO
    elem = driver.find_element(By.ID, 'total1')
    elem.send_keys(Keys.CONTROL, 'a')  # Selecciona todo
    elem.send_keys(row['TOTAL_NO_GRAVADO'])  # Sobreescribe
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_10.5
    elem = driver.find_element(By.ID, 'total2')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_10.5'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_21
    elem = driver.find_element(By.NAME, 'total3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_21'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    driver.find_element(By.ID, "Button2").click()
    time.sleep(3)

    # RAZON SOCIAL
    elem = driver.find_element(By.NAME, 'TextBox3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Cliente'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)

    # CUIT
    elem = driver.find_element(By.NAME, 'TextBox4')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['CUIT'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)   

    # Factura NÚMERO
    elem = driver.find_element(By.NAME, 'TextBox6')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Factura'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)  

    # Clase
    elem = driver.find_element(By.NAME, 'TextBox7')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['tipo3_new'])
    elem.send_keys(Keys.TAB)
    elem = driver.find_element(By.NAME, 'TextBox8')   
    elem.send_keys(row['Fecha'])
    time.sleep(1)

    # Click en Grabar
    driver.find_element(By.ID, "Button5").click()
    time.sleep(2)


KeyboardInterrupt: 

In [ ]:
# os.remove(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')